In [1]:
###########
# PRELUDE #
###########

# auto-reload changed python files
%load_ext autoreload
%autoreload 2

# nice interactive plots
%matplotlib widget

# add repository directory to include path
from pathlib import Path
import sys
PROJECT_DIR = Path('../..').resolve()
sys.path.append(str(PROJECT_DIR))

# Part 1: The Power of Two Choices

In [22]:
import matplotlib.pyplot as plt
from random import randrange, choice as randchoice
from tqdm import trange

## Goal

The goal of this part of the assignment is to gain an appreciation for the unreasonable effectiveness of simple randomized load balancing, and measure the benefits of some lightweight optimizations.

## Description
We consider random processes of the following type: there are N bins, and we throw N
balls into them, one by one. \[This is an abstraction of the sort of allocation problem that arises throughout computing—e.g. allocating tasks on servers, routing packets within parallel networks, etc..] We’ll compare four different strategies for choosing the bin in which to place a given ball.

1. Select one of the N bins uniformly at random, and place the current ball in it.

In [3]:
def choose_bin_1(N, bins):
    return randrange(N)

2. Select two of the N bins uniformly at random (either with or without replacement), and look at how many balls are already in each. If one bin has strictly fewer balls than the other, place the current ball in that bin. If both bins have the same number of balls, pick one of the two at random and place the current ball in it.

In [65]:
def choose_bin_2(N, bins):
    bin_1 = choose_bin_1(N, bins)
    bin_2 = choose_bin_1(N, bins)
    
    bin_1_size = bins[bin_1]
    bin_2_size = bins[bin_2]
    if bin_1_size == bin_2_size:
        return randchoice([bin_1, bin_2])

    elif bin_1_size < bin_2_size:
        return bin_1
    else:
        return bin_2

3. Same as the previous strategy, except choosing three bins at random rather than two.

In [5]:
def choose_bin_3(N, bins):
    bin_1 = choose_bin_1(N, bins)
    bin_2 = choose_bin_1(N, bins)
    bin_3 = choose_bin_1(N, bins)

    bin_1_size = bins[bin_1]
    bin_2_size = bins[bin_2]
    bin_3_size = bins[bin_3]

    if bin_1_size == bin_2_size == bin_3_size:
        # TODO: is this really necessary? Can't we just pick bin_1?
        return randchoice([bin_1, bin_2, bin_3])

    min_size = bin_1_size
    min_bin = bin_1
    
    if bin_2_size < min_size:
        min_size = bin_2_size
        min_bin = bin_2
    
    if bin_3_size < min_size:
        min_bin = bin_3
    
    return min_bin

4. Select two bins as follows: the first bin is selected uniformly from the first N/2 bins, and the second uniformly from the last N/2 bins. (You can assume that N is even.) If one bin has strictly fewer balls than the other, place the current ball in that bin. If both bins have the same number of balls, place the current ball (deterministically) in the first of the two bins.

In [59]:
def choose_bin_4(N, bins):
    halfN = N // 2
    bin_1 = choose_bin_1(halfN, bins)
    bin_2 = choose_bin_1(halfN, bins) + halfN
    
    bin_1_size = bins[bin_1]
    bin_2_size = bins[bin_2]
    if bin_1_size < bin_2_size:
        return bin_1
    elif bin_2_size < bin_1_size:
        return bin_2
    else:
        return bin_1
        # return randchoice([bin_1, bin_2])

(a) (5 points) Write code to simulate strategies 1–4. For each strategy, there should be a function that takes the number N of balls and bins as input, simulates a run of the corresponding random process, and outputs the number of balls in the most populated bin (denoted by X below). Before running your code, try to guess how the above schemes will compare to eachother.

In [7]:
def ball_toss(N, bin_chooser):
    """Place N balls into N bins, choosing the bin using the bin_chooser function.
    Return the maximum number of balls in any bin."""
    bins = [0] * N
    max_size = 0
    
    for _ in range(N):
        landed_bin = bin_chooser(N, bins)
        bins[landed_bin] += 1
        
        if bins[landed_bin] > max_size:
            max_size = bins[landed_bin]
    return max_size

# test each bin chooser function
ball_toss(10, choose_bin_1)
ball_toss(10, choose_bin_2)
ball_toss(10, choose_bin_3)
ball_toss(10, choose_bin_4)
"OK"

'OK'

### Hypothesis

I think 1 should do the worst job of load balancing; the nature of randomness is such that some bins will happen to be hit many times. 2 should be better, 3 even better than that. I think 4 should be equivalent to 2, since, unless our random function is not very good, there should not be any structure in the array of bins, and thus it should not help to choose them specifically from the first and second half of the array.

(b) (10 points) Let N = 200, 000 and simulate each of the four strategies 30 times. For each strategy, plot the histogram of the 30 values of X. Discuss the pros and cons of the different strategies. Does one of them stand out as a “sweet spot”? \[As with many of the mini-projects, there is no single “right answer” to this question. Instead, the idea is to have you think about the processes and your experiments, and draw reasonable conclusions from this analysis.]

In [14]:
def simulate(bin_chooser):
    max_values = []
    N = 200_000
    for _ in trange(100):
        max_values.append(ball_toss(N, bin_chooser))
    return max_values

In [16]:
max_values_1 = simulate(choose_bin_1)

100%|██████████| 100/100 [00:41<00:00,  2.41it/s]


In [66]:
max_values_2 = simulate(choose_bin_2)

100%|██████████| 100/100 [01:24<00:00,  1.19it/s]


In [20]:
max_values_3 = simulate(choose_bin_3)

100%|██████████| 100/100 [01:42<00:00,  1.02s/it]


In [60]:
max_values_4 = simulate(choose_bin_4)

100%|██████████| 100/100 [01:16<00:00,  1.31it/s]


In [67]:
data = [
    ("Method 1", max_values_1),
    ("Method 2", max_values_2), 
    ("Method 3", max_values_3), 
    ("Method 4", max_values_4)]

fig = plt.figure()
for i, d in enumerate(data):
    ax=fig.add_subplot(4, 1, i + 1)
    ax.hist(d[1], bins=range(2,10), rwidth=.5, align='left')
    ax.set_title(d[0])
    ax.set_xlabel("Maximum Bin Value", size=14)
fig.suptitle("Simulated Performance of Bin-Choosing Functions")
fig.supylabel("Simulated Frequency")
fig.tight_layout()

fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Analysis

Looks like 3 & 4 are better than 2, which is better than 1. I did not expect 4 to perform better than 2, but in retrospect it makes sense: random numbers tend to cluster, and could even be the same twice in a row! Method 4 guarantees that the 2 choices of bin are not the same, and probably also aleves the clustering issue more generally. The performance of 4 was better than 2 and 3, too, since the implementation was also simpler, so it does stand out as a possible sweet spot, assuming we can take the extra 1/3 processing time to ensure we have a good spread of bins.

I do have an open question regarding the resolution of ties. Methods 2 and 3 resolve ties via random choice; I tried deterministically choosing the first bin in method 2, but there was no change in final outcome (suggesting this is probably fine to do for time optimization). However, I also tried changing method 4 to use a random choice instead of always picking the first bin, and the final outcomes worsened, matching method 2 almost perfectly. Why would this be?

(c) (5 points) Propose an analogy between the first of the random processes above and the standard implementation of hashing N elements into a hash table with N buckets, and resolving collisions via chaining (i.e., one linked list per bucket). Discuss in particular any relationships between X and search times in the hash table.

The hypoothetical hash table performance can be compared to strategy 1 above: a pseudo-random hash function picks the bucket to put a value in, just as the function placed balls in random bins. The maximum number of balls in a bin is analogous to the number of values placed in a single bucket (in Java these are all placed in `TreeMap` when possible, otherwise a `LinkedList`). The maximum numbers in the ball-tossing simulation indicate the maximum number of iterate-and-compare steps required to find a value in the hash table.

(d) (5 points) Do the other random processes suggest alternative implementations of hash tables with chaining? Discuss the trade-offs between the different hash table implementations that you propose (e.g., in terms of insertion time vs. search time).

One could use multiple hash functions to place a value into one of multiple buckets, choosing the bucket with the fewest entries. Then the query method would search through all of the relevant buckets. The total number of iterations should still be the same on average, but the worst case performance should occur less often because we will do a better job of distributing the values. It's possible that the CPU cache behavior would be a worse, since we would access more disparate memory locations more often. Not sure about that, though.

# Part 2

## Goal
The goal of this part is to understand the count-min sketch (from Lecture #2) via an implementation, and to explore the benefits of a “conservative updates” optimization.